In [1]:
 #!nvidia-smi
import sys
sys.path.append('../')

In [2]:
from utils.cloudant_utils import cloudant_db as db
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch import nn
from collections import defaultdict
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from prepare_sequence import prepareSequenceForBERT
import random

SUCCESS connecting to Cloudant db github-public-ai-2022


/users/kent/jmaharja/anaconda3/envs/crawler/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
BATCH_SIZE = 16
MAX_LEN = 512
EPOCHS = 10
RANDOM_SEED = 42
model_name = 'bert-base-uncased'

In [ ]:
# len(new_df.iloc[1]['readme']), prepareSequenceForBERT(new_df.iloc[1]['readme'][:7])

In [23]:
repos = [r for r in db.get_query_result({"type": "release"}, ["_id", "releases"], limit=10000, raw_result=True)["docs"]]
values = [r for release in repos for r in release["releases"]]
df = pd.DataFrame(values)
df['contributors'] = df['contributors'].apply(lambda x:
                                              [i for i in x if i is not None] if isinstance(x, list)
                                              else [])
df = df[~df['readme'].isnull()]
new_df = df.groupby("repo").agg({"readme": list,
                                 "total_stars": list,
#                                  "forks": sum,
#                                  "downloads": sum,
#                                  "contributors": sum
#                                  "releases": count
                                })

In [24]:
new_df = new_df[new_df['readme'].map(len) > 1]

In [7]:
new_df = new_df[:10]

In [25]:
new_df['k'] = new_df['total_stars'].map(lambda x: random.randint(2, len(x)))

In [26]:
new_df['readme1'] = new_df.apply(lambda x: x.readme[:x.k], axis=1)

In [27]:
new_df['target_val'] = new_df.apply(lambda x: x.total_stars[x.k-1], axis=1)
new_df['target'] = new_df.apply(lambda x: 1 if x.target_val> 300 else 0, axis=1)

In [ ]:
new_df['sequence']= new_df['readme1'].apply(prepareSequenceForBERT)

In [21]:
new_df

,readme,total_stars,k,readme1,target,sequence,target_val
repo,,,,,,,
100/Solid,[# Metaheuristics-library-placeholder\n\n## Cu...,"[490, 550]",2,[# Metaheuristics-library-placeholder\n\n## Cu...,1,"[CLS], which always returns the best solution ...",550
10up/classifai,[## Klasifai\n\nClassify WordPress Content usi...,"[67, 67, 67, 67, 67, 67, 77, 77, 125, 219, 286]",6,[## Klasifai\n\nClassify WordPress Content usi...,0,"[CLS]'s,] \nSupports Watson's Categories, Keyw...",67
1adrianb/face-alignment,[# Face Recognition\n\nDetect facial landmarks...,"[3299, 3299, 4005, 4153, 4437, 4452, 4452, 445...",6,[# Face Recognition\n\nDetect facial landmarks...,1,[CLS][SEP]1.[SEP][SEP]. Support for Python 2.7...,4452
1adrianb/pytorch-estimate-flops,[# pytorch-estimate-flops\n\nSimple pytorch ut...,"[51, 51, 53, 81, 94, 98, 117, 174]",5,[# pytorch-estimate-flops\n\nSimple pytorch ut...,0,[CLS]\nIgnoring certain layers:\n```python\nim...,94
259mit/AirInteract,[# AirInteract\n\nA No-code one stop platform ...,"[3, 3, 3, 5]",3,[# AirInteract\n\nA No-code one stop platform ...,0,[CLS]\nInstallation\nClone the repo\nInstall r...,3
2Bear/othello-zero,[# othello-zero\n\n## About\n\n**othello-zero*...,"[8, 12]",2,[# othello-zero\n\n## About\n\n**othello-zero*...,0,[CLS]checkpoint File\nThe checkpoint file is j...,12
42-AI/bootcamp_machine-learning,"[<p align=""center"">\n <img src=""assets/logo_v...","[143, 146, 148, 150, 150, 152, 152, 152, 152]",4,"[<p align=""center"">\n <img src=""assets/logo_v...",0,[CLS]Download\nThe pdf files of each module ca...,150
42-AI/bootcamp_python,"[<p align=""center"">\n <img src=""assets/logo_v...","[194, 278, 284]",2,"[<p align=""center"">\n <img src=""assets/logo_v...",0,[CLS] Thanks to Ilyes and Kévin for the PR\nIl...,278
4paradigm/AutoX,[# AutoX是什么？\nAutoX一个高效的自动化机器学习工具，它主要针对于表格类型的数...,"[221, 221, 221, 221, 221, 221, 221, 221, 221, ...",13,[# AutoX是什么？\nAutoX一个高效的自动化机器学习工具，它主要针对于表格类型的数...,0,"[CLS][SEP]type | data_,(link), |, |regression,...",221


In [17]:
new_df.iloc[2]['sequence']

'[CLS][SEP]1.[SEP][SEP]. Support for Python 2.7 is deprecated.[SEP], BlazeFace,,Please check the pytorch readme for this.\nGet the Face Alignment source code\nbash\ngit clone \n,. If you plan to add a new features please open an issue to discuss this prior to making a pull request[SEP]'

In [22]:
new_df['target'].value_counts()

0    7
1    3
Name: target, dtype: int64

In [ ]:
class ReadmeDataSet(Dataset):
   def __init__(self, _df, tokenizer, max_len):
      self._df = _df
      self.tokenizer = tokenizer
      self.max_len = max_len

   def __len__(self):
      return len(self._df)

   def __getitem__(self, item):
      _sequence = self._df.iloc[item]['sequence']
      target = self._df.iloc[item]['target']
      encoding = self.tokenizer.encode_plus(_sequence,
                                     None,
                                     max_length = self.max_len,
                                     truncation=True,
                                     add_special_tokens=True,
#                                      padding=MAX_LEN,
#                                      padding='longest',
                                     pad_to_max_length=True,
                                     return_token_type_ids=True)

      return {
      'sequence': _sequence,
      'input_ids': torch.tensor(encoding.input_ids, dtype=torch.long),
      'attention_mask':  torch.tensor(encoding.attention_mask, dtype=torch.long),
      'token_type_ids': torch.tensor(encoding.token_type_ids, dtype=torch.long),
      'targets': torch.tensor(target, dtype=torch.long)
      }

In [ ]:
def create_data_loader(_df, tokenizer, max_len, batch_size):
   ds = ReadmeDataSet(_df = _df, tokenizer=tokenizer, max_len=max_len)
   return DataLoader(ds, batch_size=batch_size, num_workers=0)

Import BERT Tokenizer and BERT Model

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels=2,
                                                      output_attentions= False,
                                                      output_hidden_states= False)

In [ ]:
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

df_train, df_test = train_test_split(new_df, test_size=0.4, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)
df_train.shape, df_val.shape, df_test.shape

In [ ]:
device

In [ ]:
train_data_loader = create_data_loader(df_train, bert_tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, bert_tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, bert_tokenizer, MAX_LEN, BATCH_SIZE)
new_df.shape, df_train.shape, df_val.shape, df_test.shape

In [ ]:
for d in train_data_loader:
#     input_ids = d["input_ids"].to(device)
#     attention_mask = d["attention_mask"].to(device)
#     targets = d["targets"].to(device)
    print(d["sequence"])
    print(d["input_ids"].shape)
    print(d["targets"])
# len(val_data_loader.dataset)

In [ ]:
# TEST the tokenizer and data loader
# sequence =prepareSequenceForBERT(new_df.iloc[1]['readme'][:7])
# label = new_df.iloc[1]['total_stars'][6]
# tokens = bert_tokenizer.encode_plus(
#             sequence,
#             None,
#             max_length= 512,
#             truncation=True,
#             add_special_tokens=True,
# #             pad_to_max_length=True,
#             padding = True,
#             return_token_type_ids=True
#         )
# print(f' Sentence: {sequence}')
# print(f' Tokens: {tokens}')
# print(f' Tokens.token_type_ids: {tokens.token_type_ids}')
# print(f' Tokens.input_ids: {len(tokens.input_ids)}')
# output = {
#       'input_ids': torch.tensor(tokens.input_ids, dtype=torch.long),
#       'attention_mask':  torch.tensor(tokens.attention_mask, dtype=torch.long),
#       'token_type_ids': torch.tensor(tokens.token_type_ids, dtype=torch.long),
#       'targets': torch.tensor(label, dtype=torch.long)
#     }
# output

In [ ]:
optimizer = AdamW(bert_model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
bert_model = bert_model.to(device)

from transformers import get_linear_schedule_with_warmup
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)
# loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
def train_epoch( model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
  model = model.train()

  losses = []
  correct_predictions = 0

  for d in data_loader:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)

    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )

    _, preds = torch.max(outputs.logits, dim=1)
    loss = loss_fn(outputs.logits, targets)

    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs.logits, dim=1)

      loss = loss_fn(outputs.logits, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
%%time
history = defaultdict(list)
loss_fn = nn.CrossEntropyLoss().to(device)
best_accuracy = 0

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)

  train_acc, train_loss = train_epoch(
    bert_model,
    train_data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    len(df_train)
  )

  print(f'Train loss {train_loss} accuracy {train_acc}')

  val_acc, val_loss = eval_model(
    bert_model,
    val_data_loader,
    loss_fn,
    device,
    len(df_val)
  )

  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print()

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)

  if val_acc > best_accuracy:
    current_time = datetime.now().strftime("%Y_%m_%d-%I_%M%p")
    torch.save(bert_model.state_dict(), 'best_model_state' + current_time+'.bin')
    best_accuracy = val_acc

In [ ]:
test_acc, _ = eval_model(bert_model, test_data_loader, loss_fn, device, len(df_test))
test_acc.item()

In [ ]:
def get_predictions(model, data_loader):
  model = model.eval()

  sequences = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in data_loader:

      texts = d["sequence"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      token_type_ids = d["token_type_ids"].to(device)
      targets = d["targets"].to(device)

      outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

      _, preds = torch.max(outputs.logits, dim=1)
      probs = F.softmax(outputs.logits, dim=1)

      sequences.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(probs)
      real_values.extend(targets)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return outputs, sequences, predictions, prediction_probs, real_values

In [ ]:
outputss, y_sequences, y_pred, y_pred_probs, y_test = get_predictions(bert_model, val_data_loader)

In [ ]:
logit_roc_auc = roc_auc_score(y_test, y_pred)
fpr, tpr, thresholds = roc_curve(y_test.numpy(), y_pred_probs[:, 1].numpy())
plt.figure()
plt.plot(fpr, tpr, label=' (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC_'+ current_time +'.png')
plt.show()

In [ ]:
fpr, tpr, thresholds

In [ ]:
y_pred_probs_pd = [y.numpy() for y in y_pred_probs]
someListOfLists = list(zip(y_sequences, y_test.numpy(), y_pred.numpy(), y_pred_probs[:, 1:].numpy().squeeze(), y_pred_probs_pd ))
npa = np.asarray(someListOfLists)
dff = pd.DataFrame(someListOfLists, columns = ['readme', 'Real', 'Predicted', 'Pred-prob', 'All Pred-probs' ])
dff